# Install and Import all the dependencies

In [ ]:
! pip install validators
! pip install colorama

import sys
sys.path.append('/content/drive/My Drive/HT')
from Data.constants import SDK_PATH, DATA_PATH_HIGH_LEVEL, DATA_PATH_LABELS, ALIGNED_DATA_PATH_HIGH_LEVEL, ALIGNED_DATA_PATH_LABELS, WORD_EMB_PATH, CACHE_PATH, DATA_PATH_RAW
#from constants import SDK_PATH, DATA_PATH_HIGH_LEVEL, DATA_PATH_LABELS, ALIGNED_DATA_PATH_HIGH_LEVEL, ALIGNED_DATA_PATH_LABELS, WORD_EMB_PATH, CACHE_PATH, DATA_PATH_RAW
import os
import gc
import sys

if SDK_PATH is None:
    print("SDK path is not specified! Please specify first in constants/paths.py")
    exit(0)
else:
    sys.path.append(SDK_PATH)

if not os.path.exists(SDK_PATH):
    print("Check the relative address of SDK in the constants/paths.py, current address is: ", SDK_PATH)

if not os.path.exists(DATA_PATH_HIGH_LEVEL):
    print("Check the relative address of high level features of the data in the constants/paths.py, current address is: ", DATA_PATH_HIGH_LEVEL)

if not os.path.exists(DATA_PATH_LABELS):
    print("Check the relative address of labels of the high level features in the constants/paths.py, current address is: ", DATA_PATH_LABELS)

if not os.path.exists(ALIGNED_DATA_PATH_HIGH_LEVEL):
    print("Check the relative address of aligned high level features of the data in the constants/paths.py, current address is: ", ALIGNED_DATA_PATH_HIGH_LEVEL)

if not os.path.exists(ALIGNED_DATA_PATH_LABELS):
    print("Check the relative address of aligned labels of the high level features in the constants/paths.py, current address is: ", ALIGNED_DATA_PATH_LABELS)

if not os.path.exists(DATA_PATH_RAW):
    print("Check the relative address of raw features in the constants/paths.py, current address is: ", DATA_PATH_RAW)

import re
import math
import random
import numpy as np
from mmsdk import mmdatasdk as md

# Load the Datasets

In [ ]:
DATASET = md.cmu_mosei

data_files = os.listdir(ALIGNED_DATA_PATH_HIGH_LEVEL)
print('\n'.join(data_files))
data_files = os.listdir(DATA_PATH_RAW)
print('\n'.join(data_files))
data_files = os.listdir(ALIGNED_DATA_PATH_LABELS)
print('\n'.join(data_files))

FACET 4.2.csd
glove_vectors.csd
COAVAREP.csd
OpenSMILE.csd
OpenFace_2.0.csd
CMU_MOSEI_TimestampedWords.csd
All Labels.csd


##Only load visual features for this file

In [ ]:
visual_field = 'FACET 4.2'
acoustic_field = 'COAVAREP'
text_field = 'glove_vectors'
word_field = 'CMU_MOSEI_TimestampedWords'

features = [
    #text_field, 
    #visual_field, 
    acoustic_field
]

#raw_features = [word_field]

## Use the SDK to load the computational sequences

In [ ]:
recipe = {feat: os.path.join(ALIGNED_DATA_PATH_HIGH_LEVEL, feat) + '.csd' for feat in features}
print(recipe)

dataset = md.mmdataset(recipe)

## Inspect the data

In [ ]:
print(list(dataset.keys()))
print("=" * 80)

print(list(dataset[acoustic_field].keys())[:10])
print("=" * 80)

some_id = list(dataset[acoustic_field].keys())[15]
print(list(dataset[acoustic_field][some_id].keys()))
print("=" * 80)

word_id = list(dataset[acoustic_field].keys())[15]
print(list(dataset[acoustic_field][word_id].keys()))
print("=" * 80)
print(dataset[acoustic_field].keys())

print('Intervals')
print(list(dataset[acoustic_field][some_id]['intervals'].shape))
print("=" * 80)

print('Features')
print(list(dataset[acoustic_field][some_id]['features'].shape))

print("Different modalities have different number of time steps!")

## Load the labels

In [ ]:
label_field = 'All Labels'

# we add and align to lables to obtain labeled segments
# this time we don't apply collapse functions so that the temporal sequences are preserved
label_recipe = {label_field: os.path.join(ALIGNED_DATA_PATH_LABELS, label_field + '.csd')}
dataset.add_computational_sequences(label_recipe, destination=None)

## Split the data into train-valid-test 8:1:1

In [ ]:
identifiers = list(dataset[label_field].keys())
#print(identifiers[:10])
random.shuffle(identifiers)
print(len(identifiers))

train_index = math.floor(len(identifiers)*0.8)
dev_index = math.floor(len(identifiers)*0.9)
test_index = len(identifiers)
#print(train_index)
#print(dev_index)

## Just for testing puposes, we pick out a small cunk of samples
train_index = 100
dev_index = 120
test_index = 140

train_split = identifiers[:train_index]
dev_split = identifiers[train_index:dev_index]
test_split = identifiers[dev_index:test_index]

# inspect the splits: they only contain video IDs
print(test_split)

#Attention cell for feature extraction

## Install and Import dependecies 

In [ ]:
! pip install ray

In [ ]:
import torch
import torch.nn as nn

from tqdm import tqdm 
from collections import defaultdict
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torch.optim as optim

from ray import tune

from sklearn.metrics import classification_report

## Setup for the data loader

In [ ]:
# a sentinel epsilon for safe division, without it we will replace illegal values with a constant
EPS = 0

# place holders for the final train/dev/test dataset
train = []
dev = []
test = []

# define a regular expression to extract the video ID out of the keys
pattern = re.compile('(.*)\[.*\]')
num_drop = 0 # a counter to count how many data points went into some processing issues

In [ ]:
#for segment in dataset[label_field].keys():
for segment in identifiers[:test_index]:
    
    # get the video ID and the features out of the aligned dataset
    vid = re.search(pattern, segment).group(1)
    label = dataset[label_field][segment]['features']
    _acoustics = dataset[acoustic_field][segment]['features']

    # remove nan values
    label = np.nan_to_num(label)
    _acoustics = np.nan_to_num(_acoustics)

    print("Acoustics data shape:", _acoustics.shape, "Labels:", label.shape, "Segment ID:", segment)

    acoustics = np.asarray(_acoustics)

    # z-normalization per instance and remove nan/infs
    acoustics = np.nan_to_num((acoustics - acoustics.mean(0, keepdims=True)) / (EPS + np.std(acoustics, axis=0, keepdims=True)))

    if segment in train_split:
        ###train.append(((words, visual, acoustic), label, segment))
        train.append([acoustics, label, segment])
    elif segment in dev_split:
        ###dev.append(((words, visual, acoustic), label, segment))
        dev.append([acoustics, label, segment])
    elif segment in test_split:
        ###test.append(((words, visual, acoustic), label, segment))
        test.append([acoustics, label, segment])
    else:
        print(f"Found video that doesn't belong to any splits: {vid}")

    #input('Enter: ')

print(f"Total number of {num_drop} datapoints have been dropped.")

## Inspect the Data

In [ ]:
# let's see the size of each set and shape of data
print(len(train))
print(len(dev))
print(len(test))

print(train[0][0].shape)
print(train[0][1].shape)
print(train[0][2])

## Custom Collator to pad variable length visual sequences

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def multi_collate(batch):
    '''
    Collate functions assume batch = [Dataset[i] for i in index_set]
    '''
    # for later use we sort the batch in descending order of length
    batch = sorted(batch, key=lambda x: x[0].shape[0], reverse=True)
    
    # get the data out of the batch - use pad sequence util functions from PyTorch to pad things
    labels = torch.cat([torch.from_numpy(sample[1]).to(device) for sample in batch], dim=0)
    acoustic = pad_sequence([torch.FloatTensor(sample[0]) for sample in batch])
    
    # lengths are useful later in using RNNs
    lengths = torch.LongTensor([sample[0].shape[0] for sample in batch])

    return acoustic, labels, lengths

## Create data-loaders

In [ ]:
# construct dataloaders, dev and test could use around ~X3 times batch size since no_grad is used during eval
batch_sz = 32
train_loader = DataLoader(train, shuffle=True, batch_size=batch_sz, collate_fn=multi_collate)
dev_loader = DataLoader(dev, shuffle=False, batch_size=batch_sz*3, collate_fn=multi_collate)
test_loader = DataLoader(test, shuffle=False, batch_size=batch_sz*3, collate_fn=multi_collate)

## Checking whether the dataloader works

In [ ]:
# let's create a temporary dataloader just to see how the batch looks like
temp_loader = iter(DataLoader(train, shuffle=True, batch_size=8, collate_fn=multi_collate))
batch = next(temp_loader)

print(batch[0].shape) # acoustics vectors
print(batch[1]) # labels 
print(batch[2]) # lengths

# Model

In [ ]:
class DeltaSelfAttention(nn.Module):

    def __init__(self, in_dim, embed_dim, num_heads, dropout=0.0):
        super(DeltaSelfAttention, self).__init__()
        
        self.embedding_dimensions = embed_dim
        self.number_heads = num_heads
        self.in_dim = in_dim
        hid_dim = 1000

        self.pool = nn.AvgPool1d(2)
        self.conv = nn.Conv1d(1, 8, 5, 2)
        self.linear1 = nn.Linear(in_dim, hid_dim)
        self.linear2 = nn.Linear(hid_dim, embed_dim*2)
        
        try:
            self.mha = DeltaMultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout)
        except:
            self.mha = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout)
        
        self.key_linear = nn.Linear(embed_dim*2, embed_dim)
        self.query_linear = nn.Linear(embed_dim*2, embed_dim)
        self.value_linear = nn.Linear(embed_dim*2, embed_dim)
    
    def forward(self, inputs):
        x = self.pool(self.conv(inputs))
        x = x.reshape(x.shape[0], -1)[:, :self.in_dim]
        print(x.shape)
        x = self.linear1(x)
        print(x.shape)
        x = self.linear2(x)
        print(x.shape)
        q = self.query_linear(x).unsqueeze(0)
        k = self.key_linear(x).unsqueeze(0)
        v = self.value_linear(x).unsqueeze(0)
        print(q.shape)
        print(k.shape)
        print(v.shape)
        x = self.mha(q, k, v)
        print(x[0].shape)

        return x

## Initialize the model and training parameters

In [ ]:
model = DeltaSelfAttention(20000, 30, 3)
model = model.to(device)

epochs = 4

optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

criterion = nn.CrossEntropyLoss()

## Hyper-parameter Tuning with Ray-Tune

In [ ]:
for i in range(10):
    train(model, optimizer, train_loader)
    acc = test(model, valid_loader)
    tune.report(mean_accuracy=acc)

analysis = tune.run(train_loader, config={"lr": tune.grid_search([0.001, 0.01, 0.1])})

print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))

# Get a dataframe for analyzing trial results.
df = analysis.dataframe()

In [ ]:
torch.save(model.state_dict(), LOADER_PATH + 'acoustic_embedder')

## Train the model

In [ ]:
model = torch.load(LOADER_PATH + 'acoustic_embedder')

In [ ]:
model.train()

for epoch in range(epochs):
    print("Epoch ", epoch)

    with tqdm(train_loader, unit="batch") as tepoch:
        for inputs, labels, lengths in tepoch:
            tepoch.set_description(f"Epoch {epoch}")

            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            labels = torch.argmax(labels[:,1:], dim= 1)
            inputs = torch.transpose(inputs, 0, 1).unsqueeze(1)
            inputs = inputs.reshape(inputs.shape[0], 1, -1)

            print(inputs.shape)
            print(labels.shape)


            outputs = model(inputs)

            print(outputs[0].shape, outputs[1].shape)
            
            logits = outputs[0].squeeze(0)
            print(logits.shape)

            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            #input()

## Evaluate the model

In [ ]:
model.eval()

for epoch in range(epochs):
    print("Epoch ", epoch)

    with tqdm(test_loader, unit="batch") as tepoch:
        for inputs, labels, lengths in tepoch:
            tepoch.set_description(f"Epoch {epoch}")

            with torch.no_grad():
                inputs, labels = inputs.to(device), labels.to(device)

                labels = torch.argmax(labels[:,1:], dim= 1)
                inputs = torch.transpose(inputs, 0, 1).unsqueeze(1)
                inputs = inputs.reshape(inputs.shape[0], 1, -1)

                print(inputs.shape)
                print(labels.shape)

                outputs = model(inputs)

                print(outputs[0].shape, outputs[1].shape)
                
                logits = outputs[0].squeeze(0)
                print(logits.shape)

                loss = criterion(logits, labels)

                sm = nn.Softmax(1)
                y_pred = torch.argmax(sm(logits), 1)
                print(y_pred.shape)

                print(classification_report(labels, y_pred))

                #input()

# Function to send embeddings to other files

In [ ]:
def get_embeddings:
    return model.output_hidden_states 